# 003.4 - Historical Data Pipeline

In [1]:
import pathlib
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

sys.path.append('..')
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from src.data.utils import get_clean_season, make_interim_data, get_interim_data_fps
from src.model_selection.resampling import league_split, historical_season_split
from src.data.historical_records import form_historical_records
from src.features.custom_transformers import LastNGames, DropNthColumn, ImputeRows, \
                                            IdentityTransformer, OrderedSet, DropNaNHistories, \
                                            DropHistoriesAnyNaNFeatures

from src.metrics.rps_metrics import rps_mean

## Make interim Data and Take a Peek

### Original League Records DataFrame

In [2]:
# Get the file path list from the interim data dir
fl = get_interim_data_fps()
print(fl)

[PosixPath('/media/david/5C14F53A14F517AA/code/ana_py37_new/projects/soccer-league-models-one/data/interim/2012-2018.csv')]


In [3]:
# The file name contains the seson range 19XX-20YY.csv
# Extract a single season from the interim data
season = get_clean_season(fl[0], '2012-2013')
season.sort_values(by='date')
season.head(45)

,date,season,h,a,h_goals,a_goals,h_shots,a_shots,h_sot,a_sot,hwin_odds,draw_odds,awin_odds,result,h_phwinodds,a_pdrawodds,a_pawinodds
0,2012-08-18,2012-2013,Arsenal,Sunderland,0,0,14,3,4,2,1.44,4.72,8.71,draw,0.680082,0.207482,0.112436
1,2012-08-18,2012-2013,Fulham,Norwich,5,0,11,4,9,2,1.84,3.75,4.75,hwin,0.532471,0.261266,0.206263
2,2012-08-18,2012-2013,Newcastle,Tottenham,2,1,6,12,4,6,2.83,3.35,2.72,hwin,0.346594,0.292795,0.360611
3,2012-08-18,2012-2013,QPR,Swansea,0,5,20,12,11,8,2.00,3.53,4.15,awin,0.488162,0.276579,0.235259
4,2012-08-18,2012-2013,Reading,Stoke,1,1,9,6,3,3,2.47,3.30,3.22,draw,0.397525,0.297541,0.304934
5,2012-08-18,2012-2013,West Brom,Liverpool,3,0,15,14,10,7,4.76,3.74,1.84,hwin,0.205775,0.261895,0.532330
6,2012-08-18,2012-2013,West Ham,Aston Villa,1,0,8,10,4,6,2.14,3.50,3.71,hwin,0.456987,0.279415,0.263599
7,2012-08-19,2012-2013,Man City,Southampton,3,2,20,9,15,6,1.13,9.38,26.18,hwin,0.859378,0.103529,0.037093
8,2012-08-19,2012-2013,Wigan,Chelsea,0,2,12,5,4,3,4.92,3.77,1.81,awin,0.199073,0.259799,0.541128
9,2012-08-20,2012-2013,Everton,Man United,1,0,16,12,7,7,3.91,3.59,2.06,hwin,0.250803,0.273159,0.476039


### Convert To Historical Records

In [4]:
pd.set_option('display.max_columns', 500)
season_history = form_historical_records(season)
season_history.head(45)

,h,h_h_goals_0,h_h_goals_-1,h_h_goals_-2,h_h_goals_-3,h_h_goals_-4,h_h_goals_-5,h_h_goals_-6,h_h_goals_-7,h_h_goals_-8,h_h_goals_-9,h_h_goals_-10,h_h_goals_-11,h_h_goals_-12,h_h_goals_-13,h_h_goals_-14,h_h_goals_-15,h_h_goals_-16,h_h_goals_-17,h_h_goals_-18,h_h_shots_0,h_h_shots_-1,h_h_shots_-2,h_h_shots_-3,h_h_shots_-4,h_h_shots_-5,h_h_shots_-6,h_h_shots_-7,h_h_shots_-8,h_h_shots_-9,h_h_shots_-10,h_h_shots_-11,h_h_shots_-12,h_h_shots_-13,h_h_shots_-14,h_h_shots_-15,h_h_shots_-16,h_h_shots_-17,h_h_shots_-18,h_h_sot_0,h_h_sot_-1,h_h_sot_-2,h_h_sot_-3,h_h_sot_-4,h_h_sot_-5,h_h_sot_-6,h_h_sot_-7,h_h_sot_-8,h_h_sot_-9,h_h_sot_-10,h_h_sot_-11,h_h_sot_-12,h_h_sot_-13,h_h_sot_-14,h_h_sot_-15,h_h_sot_-16,h_h_sot_-17,h_h_sot_-18,h_h_phwinodds_0,h_h_phwinodds_-1,h_h_phwinodds_-2,h_h_phwinodds_-3,h_h_phwinodds_-4,h_h_phwinodds_-5,h_h_phwinodds_-6,h_h_phwinodds_-7,h_h_phwinodds_-8,h_h_phwinodds_-9,h_h_phwinodds_-10,h_h_phwinodds_-11,h_h_phwinodds_-12,h_h_phwinodds_-13,h_h_phwinodds_-14,h_h_phwinodds_-15,h_h_phwinodds_-16,h_h_phwinodds_-17,h_h_phwinodds_-18,h_a_goals_0,h_a_goals_-1,h_a_goals_-2,h_a_goals_-3,h_a_goals_-4,h_a_goals_-5,h_a_goals_-6,h_a_goals_-7,h_a_goals_-8,h_a_goals_-9,h_a_goals_-10,h_a_goals_-11,h_a_goals_-12,h_a_goals_-13,h_a_goals_-14,h_a_goals_-15,h_a_goals_-16,h_a_goals_-17,h_a_goals_-18,h_a_shots_0,h_a_shots_-1,h_a_shots_-2,h_a_shots_-3,h_a_shots_-4,h_a_shots_-5,h_a_shots_-6,h_a_shots_-7,h_a_shots_-8,h_a_shots_-9,h_a_shots_-10,h_a_shots_-11,h_a_shots_-12,h_a_shots_-13,h_a_shots_-14,h_a_shots_-15,h_a_shots_-16,h_a_shots_-17,h_a_shots_-18,h_a_sot_0,h_a_sot_-1,h_a_sot_-2,h_a_sot_-3,h_a_sot_-4,h_a_sot_-5,h_a_sot_-6,h_a_sot_-7,h_a_sot_-8,h_a_sot_-9,h_a_sot_-10,h_a_sot_-11,h_a_sot_-12,h_a_sot_-13,h_a_sot_-14,h_a_sot_-15,h_a_sot_-16,h_a_sot_-17,h_a_sot_-18,h_a_pdrawodds_0,h_a_pdrawodds_-1,h_a_pdrawodds_-2,h_a_pdrawodds_-3,h_a_pdrawodds_-4,h_a_pdrawodds_-5,h_a_pdrawodds_-6,h_a_pdrawodds_-7,h_a_pdrawodds_-8,h_a_pdrawodds_-9,h_a_pdrawodds_-10,h_a_pdrawodds_-11,h_a_pdrawodds_-12,h_a_pdrawodds_-13,h_a_pdrawodds_-14,h_a_pdrawodds_-15,h_a_pdrawodds_-16,h_a_pdrawodds_-17,h_a_pdrawodds_-18,h_a_pawinodds_0,h_a_pawinodds_-1,h_a_pawinodds_-2,h_a_pawinodds_-3,h_a_pawinodds_-4,h_a_pawinodds_-5,h_a_pawinodds_-6,h_a_pawinodds_-7,h_a_pawinodds_-8,h_a_pawinodds_-9,h_a_pawinodds_-10,h_a_pawinodds_-11,h_a_pawinodds_-12,h_a_pawinodds_-13,h_a_pawinodds_-14,h_a_pawinodds_-15,h_a_pawinodds_-16,h_a_pawinodds_-17,h_a_pawinodds_-18,a,a_h_goals_0,a_h_goals_-1,a_h_goals_-2,a_h_goals_-3,a_h_goals_-4,a_h_goals_-5,a_h_goals_-6,a_h_goals_-7,a_h_goals_-8,a_h_goals_-9,a_h_goals_-10,a_h_goals_-11,a_h_goals_-12,a_h_goals_-13,a_h_goals_-14,a_h_goals_-15,a_h_goals_-16,a_h_goals_-17,a_h_goals_-18,a_h_shots_0,a_h_shots_-1,a_h_shots_-2,a_h_shots_-3,a_h_shots_-4,a_h_shots_-5,a_h_shots_-6,a_h_shots_-7,a_h_shots_-8,a_h_shots_-9,a_h_shots_-10,a_h_shots_-11,a_h_shots_-12,a_h_shots_-13,a_h_shots_-14,a_h_shots_-15,a_h_shots_-16,a_h_shots_-17,a_h_shots_-18,a_h_sot_0,a_h_sot_-1,a_h_sot_-2,a_h_sot_-3,a_h_sot_-4,a_h_sot_-5,a_h_sot_-6,a_h_sot_-7,a_h_sot_-8,a_h_sot_-9,a_h_sot_-10,a_h_sot_-11,a_h_sot_-12,a_h_sot_-13,a_h_sot_-14,a_h_sot_-15,a_h_sot_-16,a_h_sot_-17,a_h_sot_-18,a_h_phwinodds_0,a_h_phwinodds_-1,a_h_phwinodds_-2,a_h_phwinodds_-3,a_h_phwinodds_-4,a_h_phwinodds_-5,a_h_phwinodds_-6,a_h_phwinodds_-7,a_h_phwinodds_-8,a_h_phwinodds_-9,a_h_phwinodds_-10,a_h_phwinodds_-11,a_h_phwinodds_-12,a_h_phwinodds_-13,a_h_phwinodds_-14,a_h_phwinodds_-15,a_h_phwinodds_-16,a_h_phwinodds_-17,a_h_phwinodds_-18,a_a_goals_0,a_a_goals_-1,a_a_goals_-2,a_a_goals_-3,a_a_goals_-4,a_a_goals_-5,a_a_goals_-6,a_a_goals_-7,a_a_goals_-8,a_a_goals_-9,a_a_goals_-10,a_a_goals_-11,a_a_goals_-12,a_a_goals_-13,a_a_goals_-14,a_a_goals_-15,a_a_goals_-16,a_a_goals_-17,a_a_goals_-18,a_a_shots_0,a_a_shots_-1,a_a_shots_-2,a_a_shots_-3,a_a_shots_-4,a_a_shots_-5,a_a_shots_-6,a_a_shots_-7,a_a_shots_-8,a_a_shots_-9,a_a_shots_-10,a_a_shots_-11,a_a_shots_-12,a_a_shots_-13,a_a_shots_-14,a_a_shots_-15,a_a_shots_-16,a_a_shot

In [5]:
season_history.tail()

,h,h_h_goals_0,h_h_goals_-1,h_h_goals_-2,h_h_goals_-3,h_h_goals_-4,h_h_goals_-5,h_h_goals_-6,h_h_goals_-7,h_h_goals_-8,h_h_goals_-9,h_h_goals_-10,h_h_goals_-11,h_h_goals_-12,h_h_goals_-13,h_h_goals_-14,h_h_goals_-15,h_h_goals_-16,h_h_goals_-17,h_h_goals_-18,h_h_shots_0,h_h_shots_-1,h_h_shots_-2,h_h_shots_-3,h_h_shots_-4,h_h_shots_-5,h_h_shots_-6,h_h_shots_-7,h_h_shots_-8,h_h_shots_-9,h_h_shots_-10,h_h_shots_-11,h_h_shots_-12,h_h_shots_-13,h_h_shots_-14,h_h_shots_-15,h_h_shots_-16,h_h_shots_-17,h_h_shots_-18,h_h_sot_0,h_h_sot_-1,h_h_sot_-2,h_h_sot_-3,h_h_sot_-4,h_h_sot_-5,h_h_sot_-6,h_h_sot_-7,h_h_sot_-8,h_h_sot_-9,h_h_sot_-10,h_h_sot_-11,h_h_sot_-12,h_h_sot_-13,h_h_sot_-14,h_h_sot_-15,h_h_sot_-16,h_h_sot_-17,h_h_sot_-18,h_h_phwinodds_0,h_h_phwinodds_-1,h_h_phwinodds_-2,h_h_phwinodds_-3,h_h_phwinodds_-4,h_h_phwinodds_-5,h_h_phwinodds_-6,h_h_phwinodds_-7,h_h_phwinodds_-8,h_h_phwinodds_-9,h_h_phwinodds_-10,h_h_phwinodds_-11,h_h_phwinodds_-12,h_h_phwinodds_-13,h_h_phwinodds_-14,h_h_phwinodds_-15,h_h_phwinodds_-16,h_h_phwinodds_-17,h_h_phwinodds_-18,h_a_goals_0,h_a_goals_-1,h_a_goals_-2,h_a_goals_-3,h_a_goals_-4,h_a_goals_-5,h_a_goals_-6,h_a_goals_-7,h_a_goals_-8,h_a_goals_-9,h_a_goals_-10,h_a_goals_-11,h_a_goals_-12,h_a_goals_-13,h_a_goals_-14,h_a_goals_-15,h_a_goals_-16,h_a_goals_-17,h_a_goals_-18,h_a_shots_0,h_a_shots_-1,h_a_shots_-2,h_a_shots_-3,h_a_shots_-4,h_a_shots_-5,h_a_shots_-6,h_a_shots_-7,h_a_shots_-8,h_a_shots_-9,h_a_shots_-10,h_a_shots_-11,h_a_shots_-12,h_a_shots_-13,h_a_shots_-14,h_a_shots_-15,h_a_shots_-16,h_a_shots_-17,h_a_shots_-18,h_a_sot_0,h_a_sot_-1,h_a_sot_-2,h_a_sot_-3,h_a_sot_-4,h_a_sot_-5,h_a_sot_-6,h_a_sot_-7,h_a_sot_-8,h_a_sot_-9,h_a_sot_-10,h_a_sot_-11,h_a_sot_-12,h_a_sot_-13,h_a_sot_-14,h_a_sot_-15,h_a_sot_-16,h_a_sot_-17,h_a_sot_-18,h_a_pdrawodds_0,h_a_pdrawodds_-1,h_a_pdrawodds_-2,h_a_pdrawodds_-3,h_a_pdrawodds_-4,h_a_pdrawodds_-5,h_a_pdrawodds_-6,h_a_pdrawodds_-7,h_a_pdrawodds_-8,h_a_pdrawodds_-9,h_a_pdrawodds_-10,h_a_pdrawodds_-11,h_a_pdrawodds_-12,h_a_pdrawodds_-13,h_a_pdrawodds_-14,h_a_pdrawodds_-15,h_a_pdrawodds_-16,h_a_pdrawodds_-17,h_a_pdrawodds_-18,h_a_pawinodds_0,h_a_pawinodds_-1,h_a_pawinodds_-2,h_a_pawinodds_-3,h_a_pawinodds_-4,h_a_pawinodds_-5,h_a_pawinodds_-6,h_a_pawinodds_-7,h_a_pawinodds_-8,h_a_pawinodds_-9,h_a_pawinodds_-10,h_a_pawinodds_-11,h_a_pawinodds_-12,h_a_pawinodds_-13,h_a_pawinodds_-14,h_a_pawinodds_-15,h_a_pawinodds_-16,h_a_pawinodds_-17,h_a_pawinodds_-18,a,a_h_goals_0,a_h_goals_-1,a_h_goals_-2,a_h_goals_-3,a_h_goals_-4,a_h_goals_-5,a_h_goals_-6,a_h_goals_-7,a_h_goals_-8,a_h_goals_-9,a_h_goals_-10,a_h_goals_-11,a_h_goals_-12,a_h_goals_-13,a_h_goals_-14,a_h_goals_-15,a_h_goals_-16,a_h_goals_-17,a_h_goals_-18,a_h_shots_0,a_h_shots_-1,a_h_shots_-2,a_h_shots_-3,a_h_shots_-4,a_h_shots_-5,a_h_shots_-6,a_h_shots_-7,a_h_shots_-8,a_h_shots_-9,a_h_shots_-10,a_h_shots_-11,a_h_shots_-12,a_h_shots_-13,a_h_shots_-14,a_h_shots_-15,a_h_shots_-16,a_h_shots_-17,a_h_shots_-18,a_h_sot_0,a_h_sot_-1,a_h_sot_-2,a_h_sot_-3,a_h_sot_-4,a_h_sot_-5,a_h_sot_-6,a_h_sot_-7,a_h_sot_-8,a_h_sot_-9,a_h_sot_-10,a_h_sot_-11,a_h_sot_-12,a_h_sot_-13,a_h_sot_-14,a_h_sot_-15,a_h_sot_-16,a_h_sot_-17,a_h_sot_-18,a_h_phwinodds_0,a_h_phwinodds_-1,a_h_phwinodds_-2,a_h_phwinodds_-3,a_h_phwinodds_-4,a_h_phwinodds_-5,a_h_phwinodds_-6,a_h_phwinodds_-7,a_h_phwinodds_-8,a_h_phwinodds_-9,a_h_phwinodds_-10,a_h_phwinodds_-11,a_h_phwinodds_-12,a_h_phwinodds_-13,a_h_phwinodds_-14,a_h_phwinodds_-15,a_h_phwinodds_-16,a_h_phwinodds_-17,a_h_phwinodds_-18,a_a_goals_0,a_a_goals_-1,a_a_goals_-2,a_a_goals_-3,a_a_goals_-4,a_a_goals_-5,a_a_goals_-6,a_a_goals_-7,a_a_goals_-8,a_a_goals_-9,a_a_goals_-10,a_a_goals_-11,a_a_goals_-12,a_a_goals_-13,a_a_goals_-14,a_a_goals_-15,a_a_goals_-16,a_a_goals_-17,a_a_goals_-18,a_a_shots_0,a_a_shots_-1,a_a_shots_-2,a_a_shots_-3,a_a_shots_-4,a_a_shots_-5,a_a_shots_-6,a_a_shots_-7,a_a_shots_-8,a_a_shots_-9,a_a_shots_-10,a_a_shots_-11,a_a_shots_-12,a_a_shots_-13,a_a_shots_-14,a_a_shots_-15,a_a_shots_-16,a_a_shot

## Custom Transformers

### Drop the 0th Record - The Current Result for this Match in the Fixture Index

In [6]:
drop_nth_col = DropNthColumn(nth_col=0)
drop_nth_col.fit(season_history)
cut_hist1 = drop_nth_col.transform(season_history)
cut_hist1.head()

,h,h_h_goals_-1,h_h_goals_-2,h_h_goals_-3,h_h_goals_-4,h_h_goals_-5,h_h_goals_-6,h_h_goals_-7,h_h_goals_-8,h_h_goals_-9,h_h_goals_-11,h_h_goals_-12,h_h_goals_-13,h_h_goals_-14,h_h_goals_-15,h_h_goals_-16,h_h_goals_-17,h_h_goals_-18,h_h_shots_-1,h_h_shots_-2,h_h_shots_-3,h_h_shots_-4,h_h_shots_-5,h_h_shots_-6,h_h_shots_-7,h_h_shots_-8,h_h_shots_-9,h_h_shots_-11,h_h_shots_-12,h_h_shots_-13,h_h_shots_-14,h_h_shots_-15,h_h_shots_-16,h_h_shots_-17,h_h_shots_-18,h_h_sot_-1,h_h_sot_-2,h_h_sot_-3,h_h_sot_-4,h_h_sot_-5,h_h_sot_-6,h_h_sot_-7,h_h_sot_-8,h_h_sot_-9,h_h_sot_-11,h_h_sot_-12,h_h_sot_-13,h_h_sot_-14,h_h_sot_-15,h_h_sot_-16,h_h_sot_-17,h_h_sot_-18,h_h_phwinodds_-1,h_h_phwinodds_-2,h_h_phwinodds_-3,h_h_phwinodds_-4,h_h_phwinodds_-5,h_h_phwinodds_-6,h_h_phwinodds_-7,h_h_phwinodds_-8,h_h_phwinodds_-9,h_h_phwinodds_-11,h_h_phwinodds_-12,h_h_phwinodds_-13,h_h_phwinodds_-14,h_h_phwinodds_-15,h_h_phwinodds_-16,h_h_phwinodds_-17,h_h_phwinodds_-18,h_a_goals_-1,h_a_goals_-2,h_a_goals_-3,h_a_goals_-4,h_a_goals_-5,h_a_goals_-6,h_a_goals_-7,h_a_goals_-8,h_a_goals_-9,h_a_goals_-11,h_a_goals_-12,h_a_goals_-13,h_a_goals_-14,h_a_goals_-15,h_a_goals_-16,h_a_goals_-17,h_a_goals_-18,h_a_shots_-1,h_a_shots_-2,h_a_shots_-3,h_a_shots_-4,h_a_shots_-5,h_a_shots_-6,h_a_shots_-7,h_a_shots_-8,h_a_shots_-9,h_a_shots_-11,h_a_shots_-12,h_a_shots_-13,h_a_shots_-14,h_a_shots_-15,h_a_shots_-16,h_a_shots_-17,h_a_shots_-18,h_a_sot_-1,h_a_sot_-2,h_a_sot_-3,h_a_sot_-4,h_a_sot_-5,h_a_sot_-6,h_a_sot_-7,h_a_sot_-8,h_a_sot_-9,h_a_sot_-11,h_a_sot_-12,h_a_sot_-13,h_a_sot_-14,h_a_sot_-15,h_a_sot_-16,h_a_sot_-17,h_a_sot_-18,h_a_pdrawodds_-1,h_a_pdrawodds_-2,h_a_pdrawodds_-3,h_a_pdrawodds_-4,h_a_pdrawodds_-5,h_a_pdrawodds_-6,h_a_pdrawodds_-7,h_a_pdrawodds_-8,h_a_pdrawodds_-9,h_a_pdrawodds_-11,h_a_pdrawodds_-12,h_a_pdrawodds_-13,h_a_pdrawodds_-14,h_a_pdrawodds_-15,h_a_pdrawodds_-16,h_a_pdrawodds_-17,h_a_pdrawodds_-18,h_a_pawinodds_-1,h_a_pawinodds_-2,h_a_pawinodds_-3,h_a_pawinodds_-4,h_a_pawinodds_-5,h_a_pawinodds_-6,h_a_pawinodds_-7,h_a_pawinodds_-8,h_a_pawinodds_-9,h_a_pawinodds_-11,h_a_pawinodds_-12,h_a_pawinodds_-13,h_a_pawinodds_-14,h_a_pawinodds_-15,h_a_pawinodds_-16,h_a_pawinodds_-17,h_a_pawinodds_-18,a,a_h_goals_-1,a_h_goals_-2,a_h_goals_-3,a_h_goals_-4,a_h_goals_-5,a_h_goals_-6,a_h_goals_-7,a_h_goals_-8,a_h_goals_-9,a_h_goals_-11,a_h_goals_-12,a_h_goals_-13,a_h_goals_-14,a_h_goals_-15,a_h_goals_-16,a_h_goals_-17,a_h_goals_-18,a_h_shots_-1,a_h_shots_-2,a_h_shots_-3,a_h_shots_-4,a_h_shots_-5,a_h_shots_-6,a_h_shots_-7,a_h_shots_-8,a_h_shots_-9,a_h_shots_-11,a_h_shots_-12,a_h_shots_-13,a_h_shots_-14,a_h_shots_-15,a_h_shots_-16,a_h_shots_-17,a_h_shots_-18,a_h_sot_-1,a_h_sot_-2,a_h_sot_-3,a_h_sot_-4,a_h_sot_-5,a_h_sot_-6,a_h_sot_-7,a_h_sot_-8,a_h_sot_-9,a_h_sot_-11,a_h_sot_-12,a_h_sot_-13,a_h_sot_-14,a_h_sot_-15,a_h_sot_-16,a_h_sot_-17,a_h_sot_-18,a_h_phwinodds_-1,a_h_phwinodds_-2,a_h_phwinodds_-3,a_h_phwinodds_-4,a_h_phwinodds_-5,a_h_phwinodds_-6,a_h_phwinodds_-7,a_h_phwinodds_-8,a_h_phwinodds_-9,a_h_phwinodds_-11,a_h_phwinodds_-12,a_h_phwinodds_-13,a_h_phwinodds_-14,a_h_phwinodds_-15,a_h_phwinodds_-16,a_h_phwinodds_-17,a_h_phwinodds_-18,a_a_goals_-1,a_a_goals_-2,a_a_goals_-3,a_a_goals_-4,a_a_goals_-5,a_a_goals_-6,a_a_goals_-7,a_a_goals_-8,a_a_goals_-9,a_a_goals_-11,a_a_goals_-12,a_a_goals_-13,a_a_goals_-14,a_a_goals_-15,a_a_goals_-16,a_a_goals_-17,a_a_goals_-18,a_a_shots_-1,a_a_shots_-2,a_a_shots_-3,a_a_shots_-4,a_a_shots_-5,a_a_shots_-6,a_a_shots_-7,a_a_shots_-8,a_a_shots_-9,a_a_shots_-11,a_a_shots_-12,a_a_shots_-13,a_a_shots_-14,a_a_shots_-15,a_a_shots_-16,a_a_shots_-17,a_a_shots_-18,a_a_sot_-1,a_a_sot_-2,a_a_sot_-3,a_a_sot_-4,a_a_sot_-5,a_a_sot_-6,a_a_sot_-7,a_a_sot_-8,a_a_sot_-9,a_a_sot_-11,a_a_sot_-12,a_a_sot_-13,a_a_sot_-14,a_a_sot_-15,a_a_sot_-16,a_a_sot_-17,a_a_sot_-18,a_a_pdrawodds_-1,a_a_pdrawodds_-2,a_a_pdrawodds_-3,a_a_pdrawodds_-4,a_a_pdrawodds_-5,a_a_pdrawodds_-6,a_a_pdrawodds_-7,a_a_pdrawodds_-8,a_a_pdrawodds_-9,a_a_pdrawodds_-11,a_a_pdrawodds_-12,a_a_pd

### Restrict the number of Historical Observations for each Feature (Get the last 3 games)

In [7]:
last_n_games = LastNGames(n_games=3)
last_n_games.fit(cut_hist1)
cut_hist2 = last_n_games.transform(cut_hist1)
cut_hist2.head()

,h_h_goals_-1,h_h_goals_-2,h_h_goals_-3,h_h_shots_-1,h_h_shots_-2,h_h_shots_-3,h_h_sot_-1,h_h_sot_-2,h_h_sot_-3,h_h_phwinodds_-1,h_h_phwinodds_-2,h_h_phwinodds_-3,h_a_goals_-1,h_a_goals_-2,h_a_goals_-3,h_a_shots_-1,h_a_shots_-2,h_a_shots_-3,h_a_sot_-1,h_a_sot_-2,h_a_sot_-3,h_a_pdrawodds_-1,h_a_pdrawodds_-2,h_a_pdrawodds_-3,h_a_pawinodds_-1,h_a_pawinodds_-2,h_a_pawinodds_-3,a_h_goals_-1,a_h_goals_-2,a_h_goals_-3,a_h_shots_-1,a_h_shots_-2,a_h_shots_-3,a_h_sot_-1,a_h_sot_-2,a_h_sot_-3,a_h_phwinodds_-1,a_h_phwinodds_-2,a_h_phwinodds_-3,a_a_goals_-1,a_a_goals_-2,a_a_goals_-3,a_a_shots_-1,a_a_shots_-2,a_a_shots_-3,a_a_sot_-1,a_a_sot_-2,a_a_sot_-3,a_a_pdrawodds_-1,a_a_pdrawodds_-2,a_a_pdrawodds_-3,a_a_pawinodds_-1,a_a_pawinodds_-2,a_a_pawinodds_-3,date,h,a
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-08-18,Arsenal,Sunderland
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-08-18,Fulham,Norwich
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-08-18,Newcastle,Tottenham
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-08-18,QPR,Swansea
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-08-18,Reading,Stoke


### Take a Look at the Feature Columns

In [8]:
feature_cols =  [col for col in cut_hist2.columns if col[-1].isdigit()]
print(feature_cols)

['h_h_goals_-1', 'h_h_goals_-2', 'h_h_goals_-3', 'h_h_shots_-1', 'h_h_shots_-2', 'h_h_shots_-3', 'h_h_sot_-1', 'h_h_sot_-2', 'h_h_sot_-3', 'h_h_phwinodds_-1', 'h_h_phwinodds_-2', 'h_h_phwinodds_-3', 'h_a_goals_-1', 'h_a_goals_-2', 'h_a_goals_-3', 'h_a_shots_-1', 'h_a_shots_-2', 'h_a_shots_-3', 'h_a_sot_-1', 'h_a_sot_-2', 'h_a_sot_-3', 'h_a_pdrawodds_-1', 'h_a_pdrawodds_-2', 'h_a_pdrawodds_-3', 'h_a_pawinodds_-1', 'h_a_pawinodds_-2', 'h_a_pawinodds_-3', 'a_h_goals_-1', 'a_h_goals_-2', 'a_h_goals_-3', 'a_h_shots_-1', 'a_h_shots_-2', 'a_h_shots_-3', 'a_h_sot_-1', 'a_h_sot_-2', 'a_h_sot_-3', 'a_h_phwinodds_-1', 'a_h_phwinodds_-2', 'a_h_phwinodds_-3', 'a_a_goals_-1', 'a_a_goals_-2', 'a_a_goals_-3', 'a_a_shots_-1', 'a_a_shots_-2', 'a_a_shots_-3', 'a_a_sot_-1', 'a_a_sot_-2', 'a_a_sot_-3', 'a_a_pdrawodds_-1', 'a_a_pdrawodds_-2', 'a_a_pdrawodds_-3', 'a_a_pawinodds_-1', 'a_a_pawinodds_-2', 'a_a_pawinodds_-3']


### Drop Fixtures where the teams have no previous historical records (played no games yet) 

In [9]:
drop_nan_histories = DropNaNHistories()
drop_nan_histories.fit(cut_hist2)
cut_hist3 = drop_nan_histories.transform(cut_hist2)
cut_hist3.head()

,h_h_goals_-1,h_h_goals_-2,h_h_goals_-3,h_h_shots_-1,h_h_shots_-2,h_h_shots_-3,h_h_sot_-1,h_h_sot_-2,h_h_sot_-3,h_h_phwinodds_-1,h_h_phwinodds_-2,h_h_phwinodds_-3,h_a_goals_-1,h_a_goals_-2,h_a_goals_-3,h_a_shots_-1,h_a_shots_-2,h_a_shots_-3,h_a_sot_-1,h_a_sot_-2,h_a_sot_-3,h_a_pdrawodds_-1,h_a_pdrawodds_-2,h_a_pdrawodds_-3,h_a_pawinodds_-1,h_a_pawinodds_-2,h_a_pawinodds_-3,a_h_goals_-1,a_h_goals_-2,a_h_goals_-3,a_h_shots_-1,a_h_shots_-2,a_h_shots_-3,a_h_sot_-1,a_h_sot_-2,a_h_sot_-3,a_h_phwinodds_-1,a_h_phwinodds_-2,a_h_phwinodds_-3,a_a_goals_-1,a_a_goals_-2,a_a_goals_-3,a_a_shots_-1,a_a_shots_-2,a_a_shots_-3,a_a_sot_-1,a_a_sot_-2,a_a_sot_-3,a_a_pdrawodds_-1,a_a_pdrawodds_-2,a_a_pdrawodds_-3,a_a_pawinodds_-1,a_a_pawinodds_-2,a_a_pawinodds_-3,date,h,a
59,1.0,2.0,0.0,18.0,9.0,12.0,9.0,5.0,4.0,0.405326,0.455999,0.199073,2.0,2.0,2.0,12.0,16.0,5.0,9.0,8.0,3.0,0.291931,0.298902,0.259799,0.302743,0.245099,0.541128,0.0,2.0,1.0,18.0,14.0,7.0,11.0,8.0,3.0,0.301765,0.301338,0.261952,3.0,0.0,3.0,30.0,12.0,19.0,19.0,5.0,11.0,0.296295,0.288893,0.285696,0.401941,0.409769,0.452352,2012-10-06,Wigan,Everton
60,1.0,2.0,3.0,9.0,14.0,15.0,4.0,8.0,10.0,0.601615,0.301338,0.205775,0.0,0.0,0.0,8.0,12.0,14.0,1.0,5.0,7.0,0.236297,0.288893,0.261895,0.162088,0.409769,0.532330,2.0,3.0,1.0,18.0,19.0,13.0,14.0,12.0,4.0,0.647739,0.829921,0.412316,1.0,1.0,1.0,9.0,9.0,6.0,6.0,5.0,4.0,0.217352,0.119573,0.271441,0.134908,0.050506,0.316242,2012-10-06,West Brom,QPR
61,1.0,3.0,1.0,20.0,17.0,8.0,11.0,12.0,4.0,0.478784,0.358078,0.456987,1.0,0.0,0.0,4.0,14.0,10.0,2.0,13.0,6.0,0.286428,0.287722,0.279415,0.234788,0.354200,0.263599,1.0,0.0,0.0,14.0,17.0,7.0,10.0,8.0,4.0,0.544995,0.520363,0.297084,1.0,2.0,0.0,10.0,11.0,16.0,4.0,7.0,6.0,0.254803,0.258805,0.287501,0.200202,0.220831,0.415415,2012-10-06,West Ham,Arsenal
62,1.0,3.0,3.0,14.0,19.0,20.0,10.0,12.0,15.0,0.544995,0.829921,0.859378,1.0,1.0,2.0,10.0,9.0,9.0,4.0,5.0,6.0,0.254803,0.119573,0.103529,0.200202,0.050506,0.037093,1.0,2.0,0.0,20.0,14.0,14.0,11.0,10.0,4.0,0.478784,0.478669,0.680082,1.0,2.0,0.0,4.0,4.0,3.0,2.0,3.0,2.0,0.286428,0.281408,0.207482,0.234788,0.239923,0.112436,2012-10-06,Man City,Sunderland
63,1.0,2.0,4.0,17.0,11.0,23.0,9.0,6.0,11.0,0.702895,0.651858,0.752297,0.0,0.0,2.0,13.0,11.0,7.0,6.0,5.0,5.0,0.198798,0.217770,0.170085,0.098307,0.130372,0.077618,1.0,1.0,5.0,16.0,15.0,11.0,8.0,9.0,9.0,0.572579,0.726707,0.532471,0.0,1.0,0.0,8.0,10.0,4.0,3.0,4.0,2.0,0.240568,0.179680,0.261266,0.186853,0.093612,0.206263,2012-10-06,Chelsea,Norwich


## Combine Transformers and League Cross Validation

In [17]:
fl = get_interim_data_fps()
season = get_clean_season(fl[0], '2012-2013')
season_history = form_historical_records(season)

pred_set_start=3
splits = historical_season_split(season_history, pred_set_start=pred_set_start)

# # Put data through Pipeline using a for Loop
feature_cols = [col for col in season_history.columns if col[-1].isdigit()]
X = season_history[feature_cols]
y = season['result']

nth_col=0
drop_nth_col = DropNthColumn(nth_col=nth_col)

n_games=3
last_n_games = LastNGames(n_games=n_games)

impute_method='ffill'
impute_rows = ImputeRows(impute_method=impute_method)

n = 0
for train_ind, val_ind in splits:
    print(train_ind)
    n = n + 1
    # Set up using the prediction set indices
    X_train, y_train, X_val, y_val = X.iloc[train_ind], y.iloc[train_ind], \
                                        X.iloc[val_ind], y.iloc[val_ind]
    drop_nth_col.fit(X_train)
    X_train1 = drop_nth_col.transform(X_train)
    
    drop_nth_col.fit(X_val)
    X_val1 = drop_nth_col.transform(X_val)
    #print(X_val1)

    last_n_games.fit(X_train1)
    X_train2 = last_n_games.transform(X_train1)
    
    last_n_games.fit(X_val1)
    X_val2 = last_n_games.transform(X_val1)
    
    impute_rows.fit(X_train2)
    X_train3 = impute_rows.transform(X_train2)
    impute_rows.fit(X_val2)
    X_val3 = impute_rows.transform(X_val2)
    if n == 1:
        break

X_val3.head()

[20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 37 38 41 42]


,h_h_goals_-1,h_h_goals_-2,h_h_goals_-3,h_h_shots_-1,h_h_shots_-2,h_h_shots_-3,h_h_sot_-1,h_h_sot_-2,h_h_sot_-3,h_h_phwinodds_-1,h_h_phwinodds_-2,h_h_phwinodds_-3,h_a_goals_-1,h_a_goals_-2,h_a_goals_-3,h_a_shots_-1,h_a_shots_-2,h_a_shots_-3,h_a_sot_-1,h_a_sot_-2,h_a_sot_-3,h_a_pdrawodds_-1,h_a_pdrawodds_-2,h_a_pdrawodds_-3,h_a_pawinodds_-1,h_a_pawinodds_-2,h_a_pawinodds_-3,a_h_goals_-1,a_h_goals_-2,a_h_goals_-3,a_h_shots_-1,a_h_shots_-2,a_h_shots_-3,a_h_sot_-1,a_h_sot_-2,a_h_sot_-3,a_h_phwinodds_-1,a_h_phwinodds_-2,a_h_phwinodds_-3,a_a_goals_-1,a_a_goals_-2,a_a_goals_-3,a_a_shots_-1,a_a_shots_-2,a_a_shots_-3,a_a_sot_-1,a_a_sot_-2,a_a_sot_-3,a_a_pdrawodds_-1,a_a_pdrawodds_-2,a_a_pdrawodds_-3,a_a_pawinodds_-1,a_a_pawinodds_-2,a_a_pawinodds_-3
39,2.0,4.0,4.0,11.0,23.0,23.0,6.0,11.0,11.0,0.651858,0.752297,0.752297,0.0,2.0,2.0,11.0,7.0,7.0,5.0,5.0,5.0,0.217770,0.170085,0.170085,0.130372,0.077618,0.077618,2.0,1.0,1.0,9.0,9.0,9.0,5.0,3.0,3.0,0.455999,0.397525,0.397525,2.0,1.0,1.0,16.0,6.0,6.0,8.0,3.0,3.0,0.298902,0.297541,0.297541,0.245099,0.304934,0.304934
40,2.0,0.0,0.0,9.0,12.0,12.0,5.0,4.0,4.0,0.455999,0.199073,0.199073,2.0,2.0,2.0,16.0,5.0,5.0,8.0,3.0,3.0,0.298902,0.259799,0.259799,0.245099,0.541128,0.541128,3.0,3.0,3.0,17.0,20.0,20.0,12.0,11.0,11.0,0.358078,0.755484,0.755484,0.0,2.0,2.0,14.0,14.0,14.0,13.0,11.0,11.0,0.287722,0.162604,0.162604,0.354200,0.081912,0.081912
43,2.0,0.0,0.0,15.0,14.0,14.0,8.0,9.0,9.0,0.135249,0.445747,0.445747,3.0,2.0,2.0,18.0,12.0,12.0,9.0,8.0,8.0,0.211949,0.282606,0.282606,0.652802,0.271646,0.271646,1.0,1.0,1.0,16.0,8.0,8.0,6.0,4.0,4.0,0.591348,0.456987,0.456987,1.0,0.0,0.0,13.0,10.0,10.0,9.0,6.0,6.0,0.248516,0.279415,0.279415,0.160137,0.263599,0.263599
44,2.0,3.0,3.0,14.0,10.0,10.0,10.0,7.0,7.0,0.478669,0.440605,0.440605,2.0,0.0,0.0,4.0,7.0,7.0,3.0,6.0,6.0,0.281408,0.287689,0.287689,0.239923,0.271706,0.271706,2.0,1.0,1.0,14.0,7.0,7.0,8.0,3.0,3.0,0.301338,0.261952,0.261952,0.0,3.0,3.0,12.0,19.0,19.0,5.0,11.0,11.0,0.288893,0.285696,0.285696,0.409769,0.452352,0.452352
45,1.0,1.0,1.0,15.0,18.0,18.0,9.0,10.0,10.0,0.726707,0.600160,0.600160,1.0,1.0,1.0,10.0,10.0,10.0,4.0,5.0,5.0,0.179680,0.234034,0.234034,0.093612,0.165807,0.165807,3.0,1.0,1.0,19.0,13.0,13.0,12.0,4.0,4.0,0.829921,0.412316,0.412316,1.0,1.0,1.0,9.0,6.0,6.0,5.0,4.0,4.0,0.119573,0.271441,0.271441,0.050506,0.316242,0.316242


In [18]:
X_val3.tail()

,h_h_goals_-1,h_h_goals_-2,h_h_goals_-3,h_h_shots_-1,h_h_shots_-2,h_h_shots_-3,h_h_sot_-1,h_h_sot_-2,h_h_sot_-3,h_h_phwinodds_-1,h_h_phwinodds_-2,h_h_phwinodds_-3,h_a_goals_-1,h_a_goals_-2,h_a_goals_-3,h_a_shots_-1,h_a_shots_-2,h_a_shots_-3,h_a_sot_-1,h_a_sot_-2,h_a_sot_-3,h_a_pdrawodds_-1,h_a_pdrawodds_-2,h_a_pdrawodds_-3,h_a_pawinodds_-1,h_a_pawinodds_-2,h_a_pawinodds_-3,a_h_goals_-1,a_h_goals_-2,a_h_goals_-3,a_h_shots_-1,a_h_shots_-2,a_h_shots_-3,a_h_sot_-1,a_h_sot_-2,a_h_sot_-3,a_h_phwinodds_-1,a_h_phwinodds_-2,a_h_phwinodds_-3,a_a_goals_-1,a_a_goals_-2,a_a_goals_-3,a_a_shots_-1,a_a_shots_-2,a_a_shots_-3,a_a_sot_-1,a_a_sot_-2,a_a_sot_-3,a_a_pdrawodds_-1,a_a_pdrawodds_-2,a_a_pdrawodds_-3,a_a_pawinodds_-1,a_a_pawinodds_-2,a_a_pawinodds_-3
46,1.0,2.0,2.0,16.0,6.0,6.0,6.0,4.0,4.0,0.591348,0.346594,0.346594,1.0,1.0,1.0,13.0,12.0,12.0,9.0,6.0,6.0,0.248516,0.292795,0.292795,0.160137,0.360611,0.360611,1.0,5.0,5.0,15.0,11.0,11.0,9.0,9.0,9.0,0.726707,0.532471,0.532471,1.0,0.0,0.0,10.0,4.0,4.0,4.0,2.0,2.0,0.179680,0.261266,0.261266,0.093612,0.206263,0.206263
47,0.0,2.0,2.0,17.0,15.0,15.0,8.0,8.0,8.0,0.520363,0.332456,0.332456,2.0,2.0,2.0,11.0,11.0,11.0,7.0,5.0,5.0,0.258805,0.285931,0.285931,0.220831,0.381613,0.381613,2.0,1.0,1.0,15.0,16.0,16.0,8.0,7.0,7.0,0.135249,0.250803,0.250803,3.0,0.0,0.0,18.0,12.0,12.0,9.0,7.0,7.0,0.211949,0.273159,0.273159,0.652802,0.476039,0.476039
48,3.0,3.0,3.0,19.0,20.0,20.0,12.0,15.0,15.0,0.829921,0.859378,0.859378,1.0,2.0,2.0,9.0,9.0,9.0,5.0,6.0,6.0,0.119573,0.103529,0.103529,0.050506,0.037093,0.037093,0.0,0.0,0.0,17.0,7.0,7.0,8.0,4.0,4.0,0.520363,0.297084,0.297084,2.0,0.0,0.0,11.0,16.0,16.0,7.0,6.0,6.0,0.258805,0.287501,0.287501,0.220831,0.415415,0.415415
52,1.0,1.0,1.0,4.0,9.0,9.0,2.0,3.0,3.0,0.266110,0.397525,0.397525,3.0,1.0,1.0,22.0,6.0,6.0,11.0,3.0,3.0,0.284676,0.297541,0.297541,0.449214,0.304934,0.304934,2.0,2.0,2.0,17.0,11.0,11.0,10.0,6.0,6.0,0.636917,0.651858,0.651858,2.0,0.0,0.0,11.0,11.0,11.0,7.0,5.0,5.0,0.225054,0.217770,0.217770,0.138029,0.130372,0.130372
53,1.0,1.0,1.0,6.0,6.0,6.0,4.0,4.0,4.0,0.266958,0.266958,0.266958,1.0,1.0,1.0,20.0,20.0,20.0,9.0,9.0,9.0,0.272163,0.272163,0.272163,0.460880,0.460880,0.460880,4.0,0.0,0.0,17.0,14.0,14.0,10.0,9.0,9.0,0.738684,0.445747,0.445747,0.0,2.0,2.0,8.0,12.0,12.0,4.0,8.0,8.0,0.172966,0.282606,0.282606,0.088350,0.271646,0.271646


In [12]:
y_train

20    hwin
21    hwin
22    draw
23    draw
24    hwin
25    draw
26    awin
27    draw
28    awin
29    draw
30    draw
31    draw
32    draw
33    hwin
34    hwin
35    hwin
37    awin
38    draw
39    awin
40    draw
41    hwin
42    hwin
43    hwin
44    awin
45    awin
46    draw
47    hwin
48    hwin
49    awin
50    hwin
51    draw
52    hwin
53    awin
54    hwin
55    awin
56    awin
57    draw
58    awin
60    hwin
63    hwin
Name: result, dtype: object